In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving geoplaces2.csv to geoplaces2.csv
Saving rating_final.csv to rating_final.csv


In [ ]:

def make_w(s):
    itr = 0
    
    w= []
    while itr!=30:
        num = 0
        for i in range(9, -1, -1):
            if s[itr] == '1':
                num+= pow(2,i)
            itr+=1
        num*=2
        num = num/1023
        w.append(num-1)
    return w

map_for_s = {}


In [ ]:
data = pd.read_csv("rating_final.csv")
names = pd.read_csv("geoplaces2.csv")
names = names.loc[:, names.columns.intersection(['placeID','name'])]
df =pd.merge(data,names,on="placeID",how='outer')
df.drop(df.columns[[3, 4]], axis=1, inplace=True)
df = df.head(500)
df.head()


,userID,placeID,rating,name
0,U1077,135085,2,Tortas Locas Hipocampo
1,U1108,135085,1,Tortas Locas Hipocampo
2,U1081,135085,1,Tortas Locas Hipocampo
3,U1056,135085,2,Tortas Locas Hipocampo
4,U1134,135085,2,Tortas Locas Hipocampo


In [ ]:
def merge_2_users(user1,user2):
    dff = df
    user2 = dff.loc[dff['userID'].isin([user2])]
    user1 = dff.loc[dff['userID'].isin([user1])]

    merged = pd.merge(user1, user2, on='placeID', suffixes=('_user1', '_user2'))
    return merged

def similarity_measure(user1,user2,s):
    merged = merge_2_users(user1,user2)
    b = len(merged)
    if b == 0:
        return 0
    
    filtered = merged[abs(merged['rating_user1'] - merged['rating_user2']) == 0]
    filtered2 = merged[abs(merged['rating_user1'] - merged['rating_user2']) == 1]
    filtered3 = merged[abs(merged['rating_user1'] - merged['rating_user2']) == 2]
    

    a1 = len(filtered['placeID'].unique())
    a2 = len(filtered2['placeID'].unique())
    a3 = len(filtered3['placeID'].unique())
    
    v = []
    v.append(a1/b)
    v.append(a2/b)
    v.append(a3/b)
    w = make_w(s)
    
    M = 2
    m = 0
    sim = w[0]*v[0]+w[1]*v[1]+w[2]*v[2]
    sim/=M-m+1
    return sim


user_ids = df['userID'].unique()

def get_similarity_matrix(s):


    
    similarity_matrix = np.zeros((len(user_ids), len(user_ids)))

   
    for i, user1 in enumerate(user_ids):
        for j, user2 in enumerate(user_ids):
            
            if j<i:
                
                similarity_matrix[i][j] = similarity_matrix[j][i]
                
            elif i == j:
                similarity_matrix[i][j] = 1
            else:
                similarity_matrix[i][j] = similarity_measure(user1, user2,s)
    
 
    return similarity_matrix




def get_k_similar_users(similarity_matrix, user_id, k):
    
    user_row = similarity_matrix[user_id]

   
    top_k_indices = np.argsort(user_row)[::-1][1:k+1]

    
    return top_k_indices.tolist()




In [ ]:
import random



K = 70


user_ids = df['userID'].unique()
unique_placeIDs = df['placeID'].unique()


def fitness(s):

    if s in map_for_s:
        return map_for_s[s]
    
    MAE =0
    
    sim_matrix = get_similarity_matrix(s)


    for i, user1 in enumerate(user_ids):
            
        sorted_data = get_k_similar_users(sim_matrix,i,K)

        user_ratings = df.loc[df['userID'].isin([user1])]
        place_ids = user_ratings['placeID'].unique().tolist()
       
        user_ratings = df.loc[(df['userID'] == user1) & (df['placeID'].isin(place_ids)), ['placeID', 'rating']]
        
        for index, row in user_ratings.iterrows():
            num =0
            deno = 0
            r_true = row['rating']

            for indx in sorted_data:
                nbrId = user_ids[indx]
                pid = row['placeID']
                filtered_df = df[(df['userID'] == nbrId) & (df['placeID'] == pid)]
                
                if len(filtered_df['rating'].values) == 0:
                    continue
                r = filtered_df['rating'].values[0]
                num+= r*sim_matrix[i][indx]
                deno+= abs(sim_matrix[i][indx])

            if deno == 0:
                continue
            r_pred = num/deno
            
            MAE += abs(r_pred-r_true)
    MAE/= len(unique_placeIDs)
    MAE/= len(user_ids)
    map_for_s[s] = MAE
    return MAE



In [ ]:

def crs_ovr(par1, par2, crs_ovr_prob):
    l = len(par1)
    if random.random() < crs_ovr_prob:

        crs_ovr_point = random.randint(1, l-1)
        chld2 = par2[:crs_ovr_point] + par1[crs_ovr_point:]
        chld2 = par1[:crs_ovr_point] + par2[crs_ovr_point:]
        
        return chld2, chld2
    else:
        return par1, par2
    
def generate_pop(pop_size, str_len):
    potential_strs = []
    for i in range(pop_size):
        s = ""
        for  j in range(str_len):
            num = random.randrange(2) 
            s += str(num)
        potential_strs.append(s)
    return potential_strs


def mutation(individual, mutation_probablity):
    mutated = ''
    for bit in individual:
        if random.random() >= mutation_probablity:
            
            mutated += bit
        else:
            mutated += '0' if bit == '1' else '1'
    return mutated


def selection(potential_strs):
    
    tournament = random.sample(potential_strs, 2)
    if fitness(tournament[0]) <= fitness(tournament[1]):
        return tournament[0]
    else:
        return tournament[1]


def next_generation(potential_strs, elite_ratio, crs_ovr_prob, mutation_probablity):
  
    potential_strs.sort(key=lambda x: fitness(x))

    elitism_size = int(len(potential_strs) * elite_ratio)
    next_gen = potential_strs[:elitism_size]

 
    while len(next_gen) < len(potential_strs):
        par2 = selection(potential_strs)
        par1 = selection(potential_strs)
        
        chld2, chld2 = crs_ovr(par1, par2, crs_ovr_prob)
        chld2 = mutation(chld2, mutation_probablity)
        chld2 = mutation(chld2, mutation_probablity)
        next_gen.append(chld2)
        next_gen.append(chld2)

    return next_gen


elite_ratio = 0.1
crs_ovr_PROB = 0.8
mutation_probablity = 0.02

potential_strs = generate_pop(10, 30)

for i in range(20):
    potential_strs = next_generation(potential_strs, elite_ratio, crs_ovr_PROB, mutation_probablity)
    best_individual = min(potential_strs, key=lambda x: fitness(x))
    print(f'Iteration {i+1}: Best string is {best_individual}, whose MAE = {fitness(best_individual)}')


Iteration 1: Best string is 111000011000100110100001000110, whose MAE = 0.007932140815843886
Iteration 2: Best string is 111000011000100110100001000110, whose MAE = 0.007932140815843886
Iteration 3: Best string is 111000011000011110100001000110, whose MAE = 0.006992167134738833
Iteration 4: Best string is 111000011000011110100001000110, whose MAE = 0.006992167134738833
Iteration 5: Best string is 111000011000011110000001001110, whose MAE = 0.004362057646611853
Iteration 6: Best string is 101000111000111110000001001110, whose MAE = 0.001974811994845046
Iteration 7: Best string is 101000111000111110000001001110, whose MAE = 0.001974811994845046
Iteration 8: Best string is 100100111000111110100001001110, whose MAE = 0.00043021357431086095
Iteration 9: Best string is 100100111000111110100001001110, whose MAE = 0.00043021357431086095
Iteration 10: Best string is 100100111000111110100001001110, whose MAE = 0.00043021357431086095
Iteration 11: Best string is 100001111000111110100001001110, wh

101010101000110000111111010110


0.69

In [8]:




def recommend_places_user_based(df_ratings, similarity_matrix, user_id, k=20):
  
    uid = user_ids[user_id]
    user_ratings = df_ratings[df_ratings['userID'] == uid].set_index('placeID')['rating']
    
   
    sim_users_indices = np.argsort(similarity_matrix[user_id])[::-1][:k]
    
  
    unrated_places = set(df_ratings['placeID'].unique()) - set(user_ratings.index)
    
   
    place_ratings = {}
    for place_id in unrated_places:
        place_sim_ratings = []
        for sim_user_id in sim_users_indices:
            uid = user_ids[sim_user_id]
            if place_id in df_ratings[df_ratings['userID'] == uid]['placeID'].values:
                sim_user_ratings = df_ratings[df_ratings['userID'] == uid].set_index('placeID')['rating']
                sim_user_sim = similarity_matrix[user_id][sim_user_id]
                place_sim_ratings.append(sim_user_sim * sim_user_ratings[place_id])
        
        if place_sim_ratings:
            place_ratings[place_id] = sum(place_sim_ratings) / len(place_sim_ratings)
    
    
    top_places = sorted(place_ratings.items(), key=lambda x: x[1], reverse=True)
    if len(top_places)>5:
        return top_places[:5]
    return top_places


sim_matrix = get_similarity_matrix("100001111100111110110001001110")

recommended_ids = recommend_places_user_based(df,sim_matrix,0)
rids = []
for plcs in recommended_ids:
    rids.append(plcs[0])


filteredP = df.loc[ (df['placeID'].isin(rids)), ['name']]
print("You should check out the following Restaurants :")
for p in filteredP['name'].unique():
    print(p)




You should check out the following Restaurants :
Gordas de morales
Mariscos El Pescador
Cafe Chaires
La Virreina
Restaurante Versalles
